In [118]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pkenken as kenken
import random
import copy

In [94]:
#Cargamos el tablero y visualizamos
input_file = r'board1.txt'
board = kenken.parse_kenken(input_file)
#kenken.plot_board(board)

In [156]:
#Generamos una respuesta

#Primero buscamos todas las permutaciones posibles dado el tablero
rango = [x+1 for x in range(board.n)]
renglones = list(itertools.permutations(rango))
len(renglones)
#print(renglones)

24

In [167]:
class Individual:
    def __init__(self,tam,permutaciones):
        self.tablero = []
        
        ind_ant = random.randint(0,tam-1)
        for _ in range(tam):
            ind_act = random.randint(0,len(permutaciones)-1)
            while(ind_act == ind_ant):
                ind_act = random.randint(0,len(permutaciones)-1) 
            self.tablero.append(permutaciones[ind_act])
            ind_ant = ind_act
        
        self.tam = tam
        self.fitness = [0,0]
        
    def crossover(self,I2):
        pcr = random.randint(1,self.tam-1)
        
        H1 = self.tablero[:pcr]
        Hh2 = I2.tablero[pcr:]
        for ren in Hh2:
            H1.append(ren)
            
    
        H2 = I2.tablero[:pcr]
        Hh1 = self.tablero[pcr:]
        for ren in Hh1:
            H2.append(ren)


        HI1=copy.deepcopy(self)
        HI2=copy.deepcopy(self)

        HI1.tablero = H1 
        HI2.tablero = H2
        return HI1,HI2

    def mutation(self,permutaciones):
        pmt = random.randint(0,self.tam-1)
        self.tablero[pmt] = permutaciones[random.randint(0,len(permutaciones)-1)]
        
    def calcfitness(self,board):
        board.load_solution(self.tablero)
        
        suma = 0
        for i in range(len(board.cages)):
            dif = board.cages[i].evaluate2() - board.cages[i].result
            if dif<0:
                dif*=-1
            suma += dif
       
        self.fitness[0]=suma
        self.fitness[1]=len(board.evaluate()[2])
        

In [168]:
def dominates(p1,p2):
    M = len(p1.fitness)
    mejor = False
    for m in range(M):
        if p1.fitness[m] > p2.fitness[m]:
            return False
        if p1.fitness[m] < p2.fitness[m]:
            mejor = True
    return mejor

In [188]:
N=20
pM = .9
G = 120
tam = board.n
Poblacion = [Individual(tam,renglones) for i in range(N)]

for i in range(G):
    for tab in Poblacion:
        tab.calcfitness(board)
    offspring = []
    for j in range(N//2):
        torneo = []
        for i in range(2):
            cont1 = Poblacion[random.randint(0,tam-1)]
            cont2 = Poblacion[random.randint(0,tam-1)]
            if(dominates(cont1,cont2)):
                torneo.append(cont1)
            elif(dominates(cont2,cont2)):
                torneo.append(cont2)
            else:
                torneo.append(cont1)
        
        H1,H2 = torneo[0].crossover(torneo[1])

        if random.random() <= pM:
            H1.mutation(renglones)
        if random.random() <= pM:
            H2.mutation(renglones)
    
        offspring.append(H1)
        offspring.append(H2)
        plebada = Poblacion + offspring
        #ordenar plebada multiobjetivo
        
        #truncar a tamaño N
        
        
print(len(Poblacion))
print(len(offspring))

20
20
